In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()
model = ChatOpenAI()

In [ ]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    feedback: str

In [ ]:
def create_outline(state: BlogState) -> BlogState:
    # fetch title
    title = state['title']
    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content
    # update state
    state['outline'] = outline
    return state

In [4]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state


In [5]:
def evaluation(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    # feedback = state['feedback']

    prompt = f'please exaluate the blog on the title - {title} and the outline {outline} out of 10, not on text only number'

    feedback = model.invoke(prompt).content 

    state['feedback'] = feedback 

    return state
    

In [6]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluation',evaluation)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluation')
graph.add_edge('evaluation', END)

workflow = graph.compile()


In [7]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': 'I. Introduction\n    A. Definition of AI \n    B. Importance of AI in India \nII. The Current Landscape of AI in India \n    A. Companies and startups leading the AI revolution \n    B. Government initiatives promoting AI technology \nIII. Applications of AI in Various Industries in India \n    A. Healthcare \n        1. AI in medical diagnosis \n        2. AI in personalized medicine \n    B. Retail \n        1. AI in customer service \n        2. AI in supply chain management \n    C. Finance \n        1. AI in fraud detection \n        2. AI in investment analysis \n    D. Agriculture \n        1. AI in crop monitoring \n        2. AI in smart agriculture \nIV. Challenges and Opportunities in the Rise of AI in India \n    A. Skill gap in the AI workforce \n    B. Ethical concerns surrounding AI technology \n    C. Opportunities for growth and innovation \nV. The Future of AI in India \n    A. Predictions for the growth of AI in India \n  

In [8]:
print(final_state['feedback'])

I would rate this blog outline a 8 out of 10. 

The outline covers all the necessary aspects of the topic "Rise of AI in India" including definitions, importance, current landscape, applications in various industries, challenges, opportunities, future predictions, and a call to action. It provides a comprehensive overview of the subject and is well-organized. 

However, to improve the outline further, it could include more specific examples of companies and startups leading the AI revolution in India, as well as delve deeper into the potential impact of AI on the Indian economy and society. Additionally, providing more detailed information on government initiatives promoting AI technology and steps to ensure responsible and ethical AI development would enhance the overall quality of the blog.
